In [10]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
SCRIPT_PATH = os.getcwd()
INPUT_DIR = os.path.abspath(os.path.join(SCRIPT_PATH, '..', 'data', 'spectrograms'))
OUTPUT_DIR = os.path.join(SCRIPT_PATH, '..', 'data', 'preprocessed', 'spectrograms')

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [8]:
# === Parameters ===
IMG_SIZE = (224, 224)   # wymagany przez VGG16
BATCH_SIZE = 16
SEED = 42
SPLIT_RATIOS = (0.7, 0.15, 0.15)

In [6]:
def split_and_save_data(source_dir, dest_dir, split_ratios):
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        images = os.listdir(cls_path)

        random.seed(SEED)
        random.shuffle(images)

        n_total = len(images)
        n_train = int(split_ratios[0] * n_total)
        n_val = int(split_ratios[1] * n_total)
        n_test = n_total - n_train - n_val

        splits = {
            "train": images[:n_train],
            "val": images[n_train:n_train+n_val],
            "test": images[n_train+n_val:]
        }

        for split_name, file_list in splits.items():
            split_cls_dir = os.path.join(dest_dir, split_name, cls)
            os.makedirs(split_cls_dir, exist_ok=True)
            for fname in file_list:
                src = os.path.join(cls_path, fname)
                dst = os.path.join(split_cls_dir, fname)
                if not os.path.exists(dst):
                    shutil.copy(src, dst)

    print(f"Data splited and saved to: {dest_dir}")

In [9]:
split_and_save_data(INPUT_DIR, OUTPUT_DIR, SPLIT_RATIOS)
train_dir = os.path.join(OUTPUT_DIR, 'train')
val_dir = os.path.join(OUTPUT_DIR, 'val')
test_dir = os.path.join(OUTPUT_DIR, 'test')

Data splited and saved to: D:\Kuba\studia\Praca_magisterska\Fault-detection-in-mechanical-devices\models_code\..\data\preprocessed\spectrograms


In [ ]:
# generators
train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=123
)

val_ds = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32,
    shuffle=False
)
test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32,
    shuffle=False
)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [ ]:
# Transfer learning from imagenet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_ds.num_classes, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint('best_vgg16_model.keras', save_best_only=True)
]

In [ ]:
# Model training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"\n Test accuracy: {test_acc*100:.2f}%")
model.save(os.join(SCRIPT_PATH,'vgg16_motor_fault_classifier_final.keras'))
print("\n💾 Model saved as: vgg16_motor_fault_classifier_final.keras")